In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stat

In [166]:
df = pd.read_csv('bank.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


### 1.모든 column을 독립변수로 활용하여 학습

In [182]:
# deposit 컬럼의 yes값을 1, no값을 0으로 변환

df['deposit'] = df['deposit'].replace({'yes':1, 'no':0})

In [183]:
X = df.drop('deposit', axis=1)
y = df['deposit']

In [184]:
categorical = [var for var in X.columns if X[var].dtype =='object']
print('categorical:',categorical)

numeric = [var for var in X.columns if X[var].dtype == 'int64']
print('numeric:',numeric)

categorical: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
numeric: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [185]:
for var in categorical:
     print(X[var].value_counts()/np.float(len(X)))

management       0.229887
blue-collar      0.174162
technician       0.163322
admin.           0.119513
services         0.082691
retired          0.069701
self-employed    0.036284
student          0.032252
unemployed       0.031984
entrepreneur     0.029385
housemaid        0.024548
unknown          0.006271
Name: job, dtype: float64
married     0.568984
single      0.315176
divorced    0.115839
Name: marital, dtype: float64
secondary    0.490593
tertiary     0.330496
primary      0.134385
unknown      0.044526
Name: education, dtype: float64
no     0.984949
yes    0.015051
Name: default, dtype: float64
no     0.526877
yes    0.473123
Name: housing, dtype: float64
no     0.869199
yes    0.130801
Name: loan, dtype: float64
cellular     0.720480
unknown      0.210177
telephone    0.069342
Name: contact, dtype: float64
may    0.253001
aug    0.136087
jul    0.135639
jun    0.109479
nov    0.084483
apr    0.082691
feb    0.069522
oct    0.035119
jan    0.030819
sep    0.028579
mar    0.0

<ipython-input-185-d3de855fa76c>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(X[var].value_counts()/np.float(len(X)))


In [186]:
for var in categorical:
    print(var, 'contains', X[var].unique(), 'labels')

job contains ['admin.' 'technician' 'services' 'management' 'retired' 'blue-collar'
 'unemployed' 'entrepreneur' 'housemaid' 'unknown' 'self-employed'
 'student'] labels
marital contains ['married' 'single' 'divorced'] labels
education contains ['secondary' 'tertiary' 'primary' 'unknown'] labels
default contains ['no' 'yes'] labels
housing contains ['yes' 'no'] labels
loan contains ['no' 'yes'] labels
contact contains ['unknown' 'cellular' 'telephone'] labels
month contains ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep'] labels
poutcome contains ['unknown' 'other' 'failure' 'success'] labels


In [187]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [188]:
# 정확도를 높이기 위해 범주형 변수들을 원-핫 인코딩처리

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()

for column in categorical:
    X[column] = label_encoder.fit_transform(X[column])

X

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,59,0,1,1,0,2343,1,0,2,5,8,1042,1,-1,0,3
1,56,0,1,1,0,45,0,0,2,5,8,1467,1,-1,0,3
2,41,9,1,1,0,1270,1,0,2,5,8,1389,1,-1,0,3
3,55,7,1,1,0,2476,1,0,2,5,8,579,1,-1,0,3
4,54,0,1,2,0,184,0,0,2,5,8,673,2,-1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,1,2,0,0,1,1,0,0,20,0,257,1,-1,0,3
11158,39,7,1,1,0,733,0,0,2,16,6,83,4,-1,0,3
11159,32,9,2,1,0,29,0,0,0,19,1,156,2,-1,0,3
11160,43,9,1,1,0,0,0,1,0,8,8,9,2,172,5,0


In [189]:
one_hot_encoder = OneHotEncoder(sparse=False, drop='first')

encoded_features = one_hot_encoder.fit_transform(X[categorical])

In [192]:
df_encoded = pd.concat([X, pd.DataFrame(encoded_features, columns=one_hot_encoder.get_feature_names(categorical))], axis=1)

df_encoded

,age,job,marital,education,default,balance,housing,loan,contact,day,...,month_5,month_6,month_7,month_8,month_9,month_10,month_11,poutcome_1,poutcome_2,poutcome_3
0,59,0,1,1,0,2343,1,0,2,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,56,0,1,1,0,45,0,0,2,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,41,9,1,1,0,1270,1,0,2,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,55,7,1,1,0,2476,1,0,2,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,54,0,1,2,0,184,0,0,2,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,1,2,0,0,1,1,0,0,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11158,39,7,1,1,0,733,0,0,2,16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11159,32,9,2,1,0,29,0,0,0,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11160,43,9,1,1,0,0,0,1,0,8,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [203]:
# 데이터를 train, test, validation set으로 분할.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_encoded, y, test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)



(7813, 51)
(1674, 51)
(1675, 51)


In [201]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
y_pred

print('Test accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Test accuracy score with 10 decision-trees : 0.8477


In [202]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train,y_train)
model_pred = clf.predict(X_val)
model_pred


print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_val, model_pred)))

Model accuracy score with 10 decision-trees : 0.8585
